# Exploratory Notebook

In [1]:
import pandas as pd
import sqlite3 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Business Question
1. What are the currrent trends by genres in the Box Office
   What genres perform best? - 
2. What budget ranges for movies are the most profitable -  to avoid spending too much, and increase ROI
3. Which actors and directors/ pull in more audiences
4. What are the average lengths of films that perform well
5. 

# 🎬 Exploratory Data Analysis (EDA): Box Office Movie Insights

## 📌 Objective
Our goal is to explore what types of films are currently performing best at the box office, in order to guide our company’s new movie studio toward data-driven content creation.

---

## 1. 🗂️ Data Overview

- Load the dataset
- Preview the first few rows
- Check shape, column names, and data types
- Identify missing values and duplicates

```python
df.shape
df.columns
df.info()
df.head()
df.isnull().sum()
df.duplicated().sum()

> I will start workin the two data sets that i have already and pick relevant data within that i can start my analysis with 

### BOM Dataset EDA
Lets us start by first having a look into the Box Office Movies Dataset and see what it looks like. 


In [10]:
df1 = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df1.info(), df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


(None,
                                          title studio  domestic_gross  \
 0                                  Toy Story 3     BV     415000000.0   
 1                   Alice in Wonderland (2010)     BV     334200000.0   
 2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
 3                                    Inception     WB     292600000.0   
 4                          Shrek Forever After   P/DW     238700000.0   
 
   foreign_gross  year  
 0     652000000  2010  
 1     691300000  2010  
 2     664300000  2010  
 3     535700000  2010  
 4     513900000  2010  )

In [12]:
# looking into df1
print(df1.info())
print("Shape of the dataset")
print(df1.shape)
print()
print("The columns on the dataset:\n", df1.columns)
print()
print("The sum of missing values", df1.isnull().sum())
print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None
Shape of the dataset
(3387, 5)

The columns on the dataset:
 Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')

The sum of missing values title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64



In [14]:
df1.head(10)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010
7,Tangled,BV,200800000.0,391000000,2010
8,Despicable Me,Uni.,251500000.0,291600000,2010
9,How to Train Your Dragon,P/DW,217600000.0,277300000,2010


###  Data Cleaning: Box Office Movies (BOM)

Before analysis, we first explore and clean the dataset to ensure it's ready for univariate, bivariate, and multivariate analyses.

---

####  Missing Values Summary

| Column          | Missing Values |
|-----------------|----------------|
| `title`         | 0              |
| `studio`        | 5              |
| `domestic_gross`| 28             |
| `foreign_gross` | 1350           |
| `year`          | 0              |

---
#### Cleaning Steps

- `studio`: Fill missing values with `"Unknown"`  
- `domestic_gross`: Drop rows with missing values (only 28 rows)  
- `foreign_gross`: Convert from string to numeric (remove commas), then handle missing as needed  
- Add new columns:
  - `worldwide_gross` = domestic + foreign
  - `proportion_foreign` = foreign / worldwide
- Drop duplicates if any

---


In [125]:
df1['title']

0                                       Toy Story 3
1                        Alice in Wonderland (2010)
2       Harry Potter and the Deathly Hallows Part 1
3                                         Inception
4                               Shrek Forever After
                           ...                     
3382                                      The Quake
3383                    Edward II (2018 re-release)
3384                                       El Pacto
3385                                       The Swan
3386                              An Actor Prepares
Name: title, Length: 3387, dtype: object

### 🎬 IMDB Database Analysis

We have an ERD (Entity Relationship Diagram) that shows the schema of our IMDB movie database. This gives us a bird’s eye view of how the data tables are related, helping us make sense of what we are working with and which parts of the database will be most useful in answering business questions.

Below is the ERD for the IMDB dataset:

![IMDB ERD Diagram](movie_data_erd.jpeg)  <!-- Adjust the path as needed -->

---

### 🧭 Next Steps for Analysis

We will follow a structured process to explore and extract value from the database:

#### 🔍 1. **Explore the Tables**
- Load all tables into Pandas DataFrames
- Inspect the shape, columns, and sample data from each

#### 🧹 2. **Clean the Data**
- Handle missing values
- Standardize data types (e.g., dates, numeric fields)
- Remove duplicates
- Engineer new features where helpful (e.g., gross revenue, genre flags)

#### 🧱 3. **Understand Relationships**
- Identify primary and foreign keys
- Decide which joins will allow us to combine useful information (e.g., `movie_basics` + `movie_ratings`)

#### 📊 4. **Perform EDA**
- Univariate: Analyze individual features like genres, years, ratings
- Bivariate: Study relationships like genre vs rating, year vs vote count
- Multivariate: Combine features to reveal deeper patterns (e.g., genre + year + rating)

#### 📈 5. **Visualize Insights**
- Use bar plots, histograms, heatmaps, and scatter plots
- Highlight patterns and trends that are directly relevant to decision-making

#### 🧠 6. **Make Business Recommendations**
- Based on data, suggest what kind of content the company should invest in
- Consider genre performance, ideal release years, director patterns, etc.

---

Let’s begin by exploring and loading all tables from the database!


In [20]:
# opening im.db file 

conn = sqlite3.connect('zippedData/im.db/im.db')
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [22]:
# Loading movie_basics
q1 = """
SELECT * 
FROM movie_basics
"""
df_movie_basics = pd.read_sql(q1, conn)
df_movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [146]:
# viewing into the movie_basics dataFrame
df_movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy
6,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller"
7,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy"
8,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History"
9,tt0144449,Nema aviona za Zagreb,Nema aviona za Zagreb,2012,82.0,Biography


In [28]:
# Loading the directors tables
q2 = """
SELECT * 
FROM persons
"""
df_directors = pd.read_sql(q2, conn)
df_directors.info(), df_directors.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
dtypes: float64(2), object(3)
memory usage: 23.1+ MB


(None,
    person_id       primary_name  birth_year  death_year  \
 0  nm0061671  Mary Ellen Bauder         NaN         NaN   
 1  nm0061865       Joseph Bauer         NaN         NaN   
 2  nm0062070         Bruce Baum         NaN         NaN   
 3  nm0062195       Axel Baumann         NaN         NaN   
 4  nm0062798        Pete Baxter         NaN         NaN   
 
                                  primary_profession  
 0         miscellaneous,production_manager,producer  
 1        composer,music_department,sound_department  
 2                        miscellaneous,actor,writer  
 3  camera_department,cinematographer,art_department  
 4  production_designer,art_department,set_decorator  )

In [30]:
df_directors.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [72]:
q3 = """
SELECT * 
FROM known_for
"""
df_4 = pd.read_sql(q3, conn)
df_4.info(), df_4.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638260 entries, 0 to 1638259
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   person_id  1638260 non-null  object
 1   movie_id   1638260 non-null  object
dtypes: object(2)
memory usage: 25.0+ MB


(None,
    person_id   movie_id
 0  nm0061671  tt0837562
 1  nm0061671  tt2398241
 2  nm0061671  tt0844471
 3  nm0061671  tt0118553
 4  nm0061865  tt0896534)

In [81]:
q4 = """
SELECT * 
FROM movie_ratings
"""
df_5 = pd.read_sql(q4, conn)
df_5.info(), df_5.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


(None,
        averagerating      numvotes
 count   73856.000000  7.385600e+04
 mean        6.332729  3.523662e+03
 std         1.474978  3.029402e+04
 min         1.000000  5.000000e+00
 25%         5.500000  1.400000e+01
 50%         6.500000  4.900000e+01
 75%         7.400000  2.820000e+02
 max        10.000000  1.841066e+06)

In [44]:
q5 = """
SELECT *
FROM movie_akas
"""

df_akas = pd.read_sql(q5, conn)

df_akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           331703 non-null  object 
 1   ordering           331703 non-null  int64  
 2   title              331703 non-null  object 
 3   region             278410 non-null  object 
 4   language           41715 non-null   object 
 5   types              168447 non-null  object 
 6   attributes         14925 non-null   object 
 7   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


In [52]:
q6 = """
SELECT *
FROM movie_ratings
"""

df_ratings = pd.read_sql(q6, conn)

df_ratings.sample(10), df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


(        movie_id  averagerating  numvotes
 3035   tt3638354            6.1         9
 2750   tt2813412            6.0        47
 27545  tt7788492            7.0        11
 22680  tt5458492            7.5        11
 54425  tt5141848            5.2        61
 15264  tt6333070            6.7      1310
 62838  tt3093522            3.8      3374
 59650  tt3757218            6.6       341
 6563   tt4603150            6.0      1021
 43626  tt5126114            8.6        10,
 None)

In [72]:
# principals table 
q7 = """
SELECT *
FROM principals
"""

df_principals = pd.read_sql(q7, conn)

df_principals.sample(10), df_principals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   movie_id    1028186 non-null  object
 1   ordering    1028186 non-null  int64 
 2   person_id   1028186 non-null  object
 3   category    1028186 non-null  object
 4   job         177684 non-null   object
 5   characters  393360 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB


(         movie_id  ordering  person_id         category       job  \
 175818  tt2149048         2  nm1436266         producer  producer   
 840238  tt8629090         2  nm0599640            actor      None   
 941832  tt6684398         5  nm8869883  cinematographer      None   
 396786  tt2585078         3  nm5082528            actor      None   
 540556  tt4236218        10  nm5902146  cinematographer      None   
 41031   tt2004191         7  nm2634191           writer      None   
 212302  tt2048889         3  nm4620003            actor      None   
 417363  tt3290440         3  nm0618427            actor      None   
 99136   tt1623182         2  nm3850604            actor      None   
 387456  tt2689660         4  nm4450035            actor      None   
 
                           characters  
 175818                          None  
 840238                          None  
 941832                          None  
 396786                     ["Henry"]  
 540556                     

**tnm.movie_budgets**

In [34]:
# Open tn_movie_budgets
df_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
df_budgets.head(), df_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


(   id  release_date                                        movie  \
 0   1  Dec 18, 2009                                       Avatar   
 1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
 2   3   Jun 7, 2019                                 Dark Phoenix   
 3   4   May 1, 2015                      Avengers: Age of Ultron   
 4   5  Dec 15, 2017            Star Wars Ep. VIII: The Last Jedi   
 
   production_budget domestic_gross worldwide_gross  
 0      $425,000,000   $760,507,625  $2,776,345,279  
 1      $410,600,000   $241,063,875  $1,045,663,875  
 2      $350,000,000    $42,762,350    $149,762,350  
 3      $330,600,000   $459,005,868  $1,403,013,963  
 4      $317,000,000   $620,181,382  $1,316,721,747  ,
 None)

In [54]:
df_budgets.sample(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
4695,96,"May 12, 1999",Trippinâ,"$3,000,000","$9,017,070","$9,017,070"
2431,32,"Jul 2, 1999",Summer of Sam,"$22,000,000","$19,288,130","$19,288,130"
3157,58,"Dec 31, 2008",Bathory,"$15,000,000",$0,"$3,436,763"
5711,12,"Mar 16, 2012",The FP,"$60,000",$0,$0
3532,33,"Feb 18, 1983",Lovesick,"$10,100,000","$10,143,618","$10,143,618"
764,65,"Jun 22, 2012",Abraham Lincoln: Vampire Hunter,"$67,500,000","$37,519,139","$137,489,730"
3868,69,"Jun 27, 2003",28 Days Laterâ¦,"$8,000,000","$45,064,915","$82,955,633"
3810,11,"Oct 25, 1991",Due occhi diabolici,"$9,000,000","$349,618","$349,618"
5713,14,"Aug 10, 1962",The Brain That Wouldn't Die,"$60,000",$0,$0
4677,78,"Jun 10, 1975",Love and Death,"$3,000,000","$20,123,742","$20,123,742"


**tmdb.movies**

In [56]:
# open tmdb.movis.csv 

df_tmdb = pd.read_csv('zippedData/tmdb.movies.csv.gz')

df_tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


**rotten tomatoes**

In [66]:
# opening rotten tomatoes movie info 

df_rt_movies = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t')

df_rt_movies.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [68]:
df_rt_movies.info(), df_rt_movies.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


(None,
 id                 0
 synopsis          62
 rating             3
 genre              8
 director         199
 writer           449
 theater_date     359
 dvd_date         359
 currency        1220
 box_office      1220
 runtime           30
 studio          1066
 dtype: int64)